# Introduction
This code simply goes through the main procedure of doing model traning, selection and evaluation. 


# Step 1: Data Input and Manipulation

In [192]:
import pprint
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series, DataFrame
import pandas as pd
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error
from statistics import mean
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import  DecisionTreeRegressor
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVR
from sklearn.linear_model import Ridge,Lasso,ElasticNet
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.metrics import recall_score, precision_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from scipy.stats import f_oneway
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.feature_selection import SelectFromModel

# get the data
import requests, zipfile
import io
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data'
res = requests.get(url).content
auto_raw = pd.read_csv(io.StringIO(res.decode('utf-8')), header=None)

# set columns
auto_raw.columns =['symboling','normalized-losses','make','fuel-type' ,'aspiration','num-of-doors',
                            'body-style','drive-wheels','engine-location','wheel-base','length','width','height',
                            'curb-weight','engine-type','num-of-cylinders','engine-size','fuel-system','bore',
                            'stroke','compression-ratio','horsepower','peak-rpm','city-mpg','highway-mpg','price']

# get abnormal values
auto_raw.isin(['?']).sum()
# therefore we need to change. replace the ? with NA (To realize this, use np.nan)
auto_raw=auto_raw.replace("?", np.nan).dropna()

# get numerical and category variable
numerical = auto_raw.select_dtypes(exclude=['object'])
category = auto_raw.select_dtypes('object')
category_copy = category[:] # This step seems important. what is the reason?
# the following four variables, although looking like 'numeric', actually are 'character'. we need to convert then into numeric.
category_copy[['horsepower', 'price', 'bore','stroke','normalized-losses','peak-rpm']] = category_copy[['horsepower', 'price', 'bore','stroke','normalized-losses','peak-rpm']].apply(pd.to_numeric)


In [154]:
auto  = pd.concat( [numerical , category_copy] , axis = 1 )
numerical_real = auto.select_dtypes (exclude = ['object'])
# turn categoryt variables into the dummy variables
category_dummy = pd.get_dummies(auto.select_dtypes('object'),drop_first=True)
auto_final = pd.concat( [numerical_real , category_dummy] , axis = 1 )
x = auto_final.drop('price',axis=1)
x_varname = x.columns
y = auto_final['price']
x

,symboling,wheel-base,length,width,height,curb-weight,engine-size,compression-ratio,city-mpg,highway-mpg,...,engine-type_ohcv,num-of-cylinders_five,num-of-cylinders_four,num-of-cylinders_six,num-of-cylinders_three,fuel-system_2bbl,fuel-system_idi,fuel-system_mfi,fuel-system_mpfi,fuel-system_spdi
3,2,99.8,176.6,66.2,54.3,2337,109,10.0,24,30,...,0,0,1,0,0,0,0,0,1,0
4,2,99.4,176.6,66.4,54.3,2824,136,8.0,18,22,...,0,1,0,0,0,0,0,0,1,0
6,1,105.8,192.7,71.4,55.7,2844,136,8.5,19,25,...,0,1,0,0,0,0,0,0,1,0
8,1,105.8,192.7,71.4,55.9,3086,131,8.3,17,20,...,0,1,0,0,0,0,0,0,1,0
10,2,101.2,176.8,64.8,54.3,2395,108,8.8,23,29,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,109.1,188.8,68.9,55.5,2952,141,9.5,23,28,...,0,0,1,0,0,0,0,0,1,0
201,-1,109.1,188.8,68.8,55.5,3049,141,8.7,19,25,...,0,0,1,0,0,0,0,0,1,0
202,-1,109.1,188.8,68.9,55.5,3012,173,8.8,18,23,...,1,0,0,1,0,0,0,0,1,0
203,-1,109.1,188.8,68.9,55.5,3217,145,23.0,26,27,...,0,0,0,1,0,0,1,0,0,0


# Step 2: Feature selection 

In [136]:
# In the following function, I try filter method and embedded method.

def feature_selection(feature_number):
    scaler = MinMaxScaler()
    scaler.fit(x)
    x_for_selection = scaler.transform (x)
    var_selected = np.empty(0)
    
    # first try fitler selection method:
    for method in [f_regression, mutual_info_regression]:
        select = SelectKBest(method, k=feature_number)
        select.fit(x_for_selection,y)
        var_selected =np.concatenate([ var_selected, x_varname[select.get_support(indices=True) ]], 0)
    
    # next try the embedded method: 
    # Lasso with l1 term can filter out the coefficient near zero. SVR does not have coef_ or feature_importance (?)
    # decision tree can output the feature importance, and can filter out those features with low importance.
    for model in [Lasso(alpha=0.1, random_state=0,max_iter = 2000),
                  RandomForestRegressor(n_estimators=100, max_leaf_nodes=16, n_jobs=-1,random_state =0 ),
                  DecisionTreeRegressor(criterion='mse', max_depth=4, random_state=0),
                 AdaBoostRegressor (n_estimators=100,learning_rate=0.5)]:
        model_fit = model.fit(x_for_selection, y)
        #lsvc = LinearSVC( penalty="l1", dual=False).fit(x, y)
        select = SelectFromModel(model_fit, prefit=True)
        var_selected =np.concatenate([ var_selected, x_varname[select.get_support(indices=True) ]], 0)
    
    # now calculate the frequency of each variable in var_selected
    unique_elements, counts_elements = np.unique(var_selected, return_counts=True)
    counts_elements,unique_elements = (list(t) for t in zip(*sorted(zip( list(counts_elements), list(unique_elements) ))))
    unique_elements.reverse()
    counts_elements.reverse()
    return unique_elements[0:feature_number],counts_elements[0:feature_number]
    

In [163]:
feature_name = feature_selection(8) [0]
# finally we finished with the selection.
x_final = x.loc[:,x.columns.isin(feature_name)]
scaler = MinMaxScaler()
scaler.fit(x_final)
x_final = scaler.transform (x_final)

# Step 3: Model Selection via hyperparameter tuning
We next use GridsearchCV in sklearn to do hyperparameter tuning for each type algorithm. We consider the following algorithms: random forest, lasso regression, adaboost, gradient boost, and svr
<p>the following links summarizes some general rule of thumbs when tuning parameters.
https://qiita.com/R1ck29/items/50ba7fa5afa49e334a8f </p>

In [170]:
# we use this function to perform hyperparameter tuning for a given algorithm.
# take parameter grid, model, and scoring as input, the function outputs the 'best' model with best score
def grid_search (param_grid, clf, score):
    # do not use repeatedstratifiedkfold heare.
    my_cv= RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)
    grid_search = GridSearchCV(clf, param_grid, cv=my_cv,scoring=score,return_train_score=True)
    grid_search.fit(x_final, y)
    return grid_search.best_estimator_,grid_search.best_score_,grid_search.cv_results_['std_test_score'][grid_search.best_index_] 

In [171]:
# The following function get best model from each algorithm 
# through these methods, I can get the votes for each features. I select those features with most votes
def ModelSelection (criterion):
    score = criterion
    
    # we first fine tune randomforest 
    param_grid = [{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},{'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
     ]
    clf = RandomForestRegressor() 
    rdf_best = grid_search (param_grid, clf, score)

    # we next do svr 
    param_grid = [
     { 'C': [0.1, 0.3, 0.6, 1], 'kernel': ['linear','rbf']},
     ]
    clf = SVR() 
    svr_best = grid_search (param_grid, clf, score) 

    # we next fine tune adaboost
    # gridsearchCV is also useful for ensemble learning. 
    # here, notice that you may want to tune parameters both for the esembler (learning rate, number of estimators, and etc..) and the parameter 
    # for the the base estimator. you can use 'base_estimator__HyperparameterName' to tell sklearn that you are specifying the hyperparameter of base model.
    # see https://stackoverflow.com/questions/32210569/using-gridsearchcv-with-adaboost-and-decisiontreeclassifier
    # This is also time consuming. 
    param_grid = [
     { 'base_estimator__max_depth': [1,2,3,4],'n_estimators':[50,75,100,125],'learning_rate':[0.1,0.3,0.5]}
     ]
    dtr = DecisionTreeRegressor()
    clf = AdaBoostRegressor(base_estimator = dtr) 
    adb_best = grid_search (param_grid, clf, score) 

    # We next fine-tune the gradientboost. 
    # notice that gradient boost use decision tree as base estimator, so its hyperparameters naturally contains the hyperparameters of tree.
    # therefore there is no worry about the specification of 'base_estimator'
    param_grid = [
     { 'max_depth': [1,3,5],'n_estimators':[50,75,100,125],'learning_rate':[0.05,0.1,0.15,0.2]}
     ]
    clf = GradientBoostingRegressor() 
    gdb_best = grid_search (param_grid, clf, score) 
    
    # Finally we fine tune the lasso
    param_grid = {
    'alpha' : [  0.25, 0.5, 0.75,1]
    }
    ls = Lasso(max_iter = 2000)
    clf = Lasso()
    ls_best = grid_search (param_grid, clf, score)
    
    # Here we do not consider svc, since it is super time consuming. In practice, however, we can use RandomizedSearchCV to choose the best
    # hyperparameters for svc
    
    return rdf_best, svr_best,adb_best, gdb_best, ls_best

In [199]:
model_result = ModelSelection('r2')
# show the results in data frame
pd.DataFrame(model_result,columns=['MODEL', 'performance_mean', 'performance_std'])

,MODEL,performance_mean,performance_std
0,"(DecisionTreeRegressor(max_features=2, random_...",0.886084,0.043629
1,"SVR(C=1, kernel='linear')",-0.159987,0.088055
2,"(DecisionTreeRegressor(max_depth=4, random_sta...",0.859909,0.044611
3,([DecisionTreeRegressor(criterion='friedman_ms...,0.873962,0.032291
4,Lasso(alpha=1),0.766220,0.078314


# Step 4: Model(Algorithm) Comparisons using statistical tests
We now turn to statistical tests to compare the models we selected from stage 3.
<p>
main reference: Chapter 6 in 'evaluating learning algorithms a classification perspective'</p>

### a.Comparing two algorithms using 5*2 cv
In general, 
1. if you evaluate the performance of two classifiers over multiple domain, then 5*2 cv is a good choice. Paired t test via resampling is also a method, but often times the assumption for paired t test is violated. 
2. If you evaluate the performance of two classifier over one domain, then Mc Nemar contingency table test is a good choice.

In [208]:
# for comparing two regressors using 5*2 cv
# see https://machinelearningmastery.com/hypothesis-test-for-comparing-machine-learning-algorithms/
# Also see http://rasbt.github.io/mlxtend/user_guide/evaluate/paired_ttest_5x2cv/
from mlxtend.evaluate import paired_ttest_5x2cv
t, p = paired_ttest_5x2cv(estimator1=clf1,estimator2=clf2,X=x_final, y=y,random_seed=1)
ttest_result =[ [model_result[i][0], model_result[j][0],   paired_ttest_5x2cv(estimator1=model_result[i][0],estimator2=model_result[j][0],X=x_final, y=y,random_seed=1)[1]] \
for i in range(0,len(model_result)-1)  for j in range(i+1,len(model_result)) ]
ttest_table = pd.DataFrame(np.vstack(ttest_result), columns= ['model1','model2','p-value'])
ttest_table
# however, pair wise comparison using t test may be misleading sometimes. 

,model1,model2,p-value
0,"(DecisionTreeRegressor(max_features=2, random_...","SVR(C=1, kernel='linear')",0.00108453
1,"(DecisionTreeRegressor(max_features=2, random_...","(DecisionTreeRegressor(max_depth=4, random_sta...",0.699242
2,"(DecisionTreeRegressor(max_features=2, random_...",([DecisionTreeRegressor(criterion='friedman_ms...,0.824972
3,"(DecisionTreeRegressor(max_features=2, random_...",Lasso(alpha=1),0.0241685
4,"SVR(C=1, kernel='linear')","(DecisionTreeRegressor(max_depth=4, random_sta...",0.00190213
5,"SVR(C=1, kernel='linear')",([DecisionTreeRegressor(criterion='friedman_ms...,0.00223631
6,"SVR(C=1, kernel='linear')",Lasso(alpha=1),0.0044257
7,"(DecisionTreeRegressor(max_depth=4, random_sta...",([DecisionTreeRegressor(criterion='friedman_ms...,0.621279
8,"(DecisionTreeRegressor(max_depth=4, random_sta...",Lasso(alpha=1),0.0267478
9,([DecisionTreeRegressor(criterion='friedman_ms...,Lasso(alpha=1),0.00402075


### b.Comparing multiple algorithms 
The following code block is the functions for multiple classifier comparison and post-hoc test.
1. Parametric method: ANOVA, followed by Tukey test as post-hoc test. 
2. Nonparametric method: Friedmand test, followed by Nemenyi test as post-hoc test.



In [188]:
# Notice : I only output the statisitics. need to check the threhold value table to determine whether to reject or accept the null hypothesis.
#The algorithm is based on the 6.7.1 of 'evaluating learning algorithms a classification perspective'
def multiple_clf_compare_anova(clf_list,n):
    k = len (clf_list)
    my_cv = KFold(n_splits=n, shuffle = False)
    pfm = [ cross_val_score (model, x_final, y, cv = my_cv) for model in clf_list  ]
    pfm_table = pd.DataFrame(np.vstack(pfm)).T
    # we get the results for ANOVA test. 
    # anova = f_oneway(*pfm)
    pfm_stack = pfm_table.stack ()
    ss_total = sum([(x-mean(pfm_stack ))**2 for x in pfm_stack])
    ss_pfm =   n* sum ([(x-mean(pfm_stack))**2   for x in  pfm_table.mean (axis = 0)])
    ss_block = k * sum ([(x-mean(pfm_stack))**2   for x in  pfm_table.mean (axis = 1)])
    ss_error = ss_total - (ss_pfm + ss_block)
    ms_pfm , ms_error = ss_pfm /(k-1) , ss_error /((n-1)*(k-1))
    F = ms_pfm / ms_error
    return F,pfm_table, ms_error

# The algorithm is based on the 6.7.5 of 'evaluating learning algorithms a classification perspective'
def multiple_clf_compare_friedman(clf_list,n):
    my_cv = KFold(n_splits=n, shuffle = False)
    pfm = [ cross_val_score (model, x_final, y, cv = my_cv) for model in clf_list  ]
    # we get results from the friedman test 
    # please check chapter 6 of 
    pfm_table = pd.DataFrame(np.vstack(pfm)).T
    rank_table = pfm_table.rank(1, ascending=False, method='average')
    k = len (clf_list)
    friedman = 12* (np.square (rank_table.sum(axis = 0) ).sum())/(n * k * (k+1) )- 3 * n * (k +1)
    return friedman,rank_table

# if the friedman test shows that the there is difference between multiple classifiers, then do the post-doc test
#  The algorithm is based on the 6.7.7 of 'evaluating learning algorithms a classification perspective'
def post_hoc_anova (pfm_table,ms_error):
    n,k =pfm_table.shape
    avg_pfm = pfm_table.mean (axis = 0)
    divide = (ms_error/n)**0.5
    df = (n-1)*(k-1)
    return [ [i,j, (avg_pfm[i]-avg_pfm[j])/ divide ]  for i in range (0,k-1) for j in range (i+1,k)], df,k

#  The algorithm is based on the 6.7.8 of 'evaluating learning algorithms a classification perspective'
def post_hoc_friedman(rank_table):
    n,k =rank_table.shape
    avg_rank = rank_table.mean(axis = 0)  # for each classifier, get its average rank
    divide =( k*(k+1)/(6*n) )**0.5
    df = (n-1)*(k-1)
    return [ [i,j, (avg_rank[i]-avg_rank[j])/ divide ]  for i in range (0,k-1) for j in range (i+1,k)], df,k
    # the degree of freedom is (n-1)*(k-1)

In [190]:
clf_list =[x[0] for x in model_result]
a=multiple_clf_compare_friedman (clf_list,10)
print('the chi2 statistic is',a[0])

(15.840000000000003,
      0    1    2    3    4
 0  2.0  5.0  1.0  3.0  4.0
 1  3.0  5.0  2.0  4.0  1.0
 2  4.0  5.0  3.0  1.0  2.0
 3  2.0  5.0  1.0  3.0  4.0
 4  4.0  5.0  1.0  2.0  3.0
 5  4.0  5.0  2.0  3.0  1.0
 6  1.0  5.0  3.0  2.0  4.0
 7  1.0  2.0  4.0  5.0  3.0
 8  3.0  5.0  1.0  2.0  4.0
 9  2.0  5.0  3.0  4.0  1.0)

## Some Appendix 
### A. More on the Feature selection method

### Method 1 : Filter Method
I can use feature_selection library from sklearn. class SelectKBest provides the tool to find out the k 'best' variables, in the sense that they have the highest correlation with target. Since I am dealing with binary target, I can use chi2, mutual_info_classif (mutual infomration) or f_classif (ANOVA) criteria to select best feature .
see https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection

In [61]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import  f_regression, mutual_info_regression
# write a simple function that, use method and desired feature number as input
# output the x_new (feature data) and x_varname_selected (feature name)
def featureSelect (method_input, number_of_feature):
    select = SelectKBest(method_input, k=number_of_feature)
    x_new = select.fit_transform(x_final, y)
    x_varname_selected =x_varname[select.get_support(indices=True) ]
    return x_new, x_varname_selected


In [63]:
a= featureSelect(mutual_info_classif, 8)
list(a[1])

['age',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'marital-status_ Married-civ-spouse',
 'marital-status_ Never-married',
 'relationship_ Own-child']

### Method 2: Recursive elimination
Given an external estimator that assigns weights to features (e.g., the coefficients of a linear model), recursive feature elimination (RFE) is to select features by recursively considering smaller and smaller sets of features. First, the estimator is trained on the initial set of features and the importance of each feature is obtained either through a coef_ attribute or through a feature_importances_ attribute. Then, the least important features are pruned from current set of features.That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached.
Although this method works during the data training, we can still use this method to precheck the potential good features. 
however, when specifying the model, we must specify the hyperparameters. Therefore, we can actually choose number of features and model hyperparameters simultaneously. 
This method is super time consuming

In [39]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
# This is super time-consuming 
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE
svc = SVC(kernel="linear", C=1)
selector = RFE(svc, n_features_to_select=8, step=2)
selector.fit(x,y)
selector.support_


array([False, False,  True,  True,  True, False, False, False, False,
       False, False, False, False, False,  True,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False])

### Method 3: Select from model (Embedded Method)

<p>Here are some 
the following link is clear and clean.
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html#sklearn.feature_selection.SelectFromModel </p>



#### a.Model selection using the L1 regularization term
remember in the textbook that l1 regularization term can be used for feature selection, since it can generate sparse coefficient matrix (many coefficients will turn to zero.) we can get rid of these features with zero coefficient. Therefore, theoretically, any model that can apply l1 regularization can be used for feature selection.
<p>
In SVC, logistic we can specify the l1 penalty (in logistic we need to specify the solver as saga). according to sklearn document, we can also set the value of C, which is the inverse of the strength of regularization. the smaller C the fewer features selected'.<\p>
        <p>
    When we set penalty to l1 and perform selectfrommodel, the default threshold is 1e-5. otherwise, the default threhold is mean(abs(coef)). I recommend use C to control. </p>
In lasso, the model automatically has l1. the default threshold is therefore 1e-5, we can use these 3 models to select feature (since here I am dealing with classification, it is better to use svc or logistics).
With Lasso, the higher the alpha parameter, the fewer features selected.

In [44]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
#lr = LogisticRegression( penalty="l1", solver ='saga').fit(x, y)
lr = Lasso( alpha=0.3).fit(x, y)
#lsvc = LinearSVC( penalty="l1", dual=False).fit(x, y)
select = SelectFromModel(lr, prefit=True)
x_varname_selected =x_varname[select.get_support(indices=True) ]
# use lr.coef_ and select.get_support(indices=True) to check the selection

#### b. Feature selection using feature importance in decision tree
Remember that Tree-based estimators (see the sklearn.tree module and forest of trees in the sklearn.ensemble module) can be used to compute impurity-based feature importances, which in turn can be used to discard irrelevant features (when coupled with the sklearn.feature_selection.SelectFromModel meta-transformer)
<p> This implies that in general,tree type classifier can be used for feature selection based on feature importance score they output. Some commonly used classifiers are random forest, decision tree, adaboost with decision stumps. See the following link for more inoformation: https://stats.stackexchange.com/questions/51676/using-adaboost-for-feature-selection </p>


In [58]:

tree = RandomForestRegressor(n_estimators=100, max_leaf_nodes=16, n_jobs=-1).fit(x,y)
#tree = DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=0).fit(x,y)
select = SelectFromModel(tree, prefit=True)
x_varname_selected =x_varname[select.get_support(indices=True)]

## B. Choosing the best number of base clfs for iterative ensemble method
For iterative ensemble method such as adaboost and gradient boost, you can use early stopping to choose the best number of base classifiers.

In [16]:

def try_boost(model):
    skfolds = KFold(n_splits=5) 
    best_n,min_error = [],[]
    for train_index, test_index in skfolds.split(x, y):
    # for each split, we train a gradient boost and uses early stopping to find the best number of estimator
        x_train, y_train = x[train_index],y[train_index]
        x_test, y_test =x[test_index], y[test_index]
        model.fit(x_train, y_train)
        errors = [mean_squared_error(y_test, y_pred) for y_pred in model.staged_predict(x_test)]
        best_n.append ( np.argmin(errors) + 1 )
        min_error.append (min(errors))
    return round(mean(best_n))+1, 1 - mean(min_error)